## Creating a program in the IR of the stam language

First, let's import the necessary imports.

In [24]:
from hw.cpu.assembler import asm
from sw.compiler.backend import CompilerBackend
from sw.compiler.parser import IRParser

p = IRParser()
cb = CompilerBackend()

Now let's write a function in StamlIR that draws a rectangle (we'll also add the horiz and vert functions given to us as examples):

In [31]:
cb.funcs(p('''
/* horiz x0 x1 y  */
horiz(3) =
    (if $2 < $1 then 1 else (if $1 < $2 then 0xffff else 0));
    if $-1 then ignore block $2 $0; ignore wait; .horiz ($2 + $-1) $1 $0
           else 0

/* vert x y0 y1  */
vert(3) =
    (if $1 < $0 then 1 else (if $0 < $1 then 0xffff else 0));
    if $-1 then ignore block $2 $1; ignore wait; .vert $2 ($1 + $-1) $0
           else 0

/* square x y len  */
square(3) =
    ignore horiz $0 ($0 + $2) $1;  /* top line */
    ignore vert ($0 + $2) $1 ($1 + $2);  /* right line */
    ignore horiz ($0 + $2) $0 ($1 + $2);  /* bottom line */
    ignore vert $0 ($1 + $2) $1;  /* left line */
    0

main(0) =
    ignore square 5 5 10  /* top line */
'''))
cb.code

['horiz',
 ('DUP', 2),
 ('DUP', 2),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'horiz:0'),
 ('PUSH', 1),
 ('JMP', 'horiz:1'),
 'horiz:0',
 ('DUP', 1),
 ('DUP', 3),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'horiz:2'),
 ('PUSH', 65535),
 ('JMP', 'horiz:3'),
 'horiz:2',
 ('PUSH', 0),
 'horiz:3',
 'horiz:1',
 ('DUP', 0),
 ('POP', 1),
 ('JZ', 'horiz:4'),
 ('PUSH', 'horiz:5'),
 ('DUP', 4),
 ('DUP', 3),
 ('JMP', 'block'),
 'horiz:5',
 ('POP', 1),
 ('PUSH', 'horiz:6'),
 ('JMP', 'wait'),
 'horiz:6',
 ('POP', 1),
 ('DUP', 3),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'ADD'),
 ('DUP', 3),
 ('DUP', 3),
 ('YANK', (3, 4)),
 ('JMP', 'horiz'),
 ('JMP', 'horiz:7'),
 'horiz:4',
 ('PUSH', 0),
 'horiz:7',
 ('YANK', (1, 1)),
 ('YANK', (1, 3)),
 ('POP', 2),
 ('RET', 1),
 'vert',
 ('DUP', 1),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'vert:0'),
 ('PUSH', 1),
 ('JMP', 'vert:1'),
 'vert:0',
 ('DUP', 0),
 ('DUP', 2),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'vert:2'),
 ('P

In [2]:

# prog = [
#     PUSH 0xffff
#     PUSH 0xa000
#     POP 2
#     STOR
#     PUSH 0xffff
#     PUSH 0xb000
#     POP 2
#     STOR
#     PUSH 0xffff
#     PUSH 0xc000
#     POP 2
#     STOR 
# ]

In [10]:
# define a function that draws a square in IR code
print(cb.funcs(p('''
    write_pixel(3) =
        mem_poke $1 
''')))
cb.code

ParseException: Expected end of text, found 'square'  (at char 1), (line:2, col:1)